<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Imports
import pandas as pd
import numpy as np
import collections

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Defining the starting dataframe
df = pd.read_csv('Personnel names - Sheet1.csv')
df = df.rename(columns={'Unnamed: 2':'Working?','Unnamed: 3':'Hours Worked'}) 
df['Working?'] = False
df['Hours Worked'] = 0
df['Where?'] = 'Taunton, Massachusetts'
df['Energy'] = 12
df['Status'] = 'Worker'
df.loc[0:24,'Status'] = 'Pilot'
groups = dict()

#Defining resources dataframe
resources = pd.read_csv('Resources CSV - Sheet2.csv')

vehicles = pd.read_csv('Resources CSV - Sheet1.csv')

assets = pd.read_csv('Resources CSV - Sheet3.csv')

facilities = pd.read_csv('facilities - Sheet1.csv')
facilities['Running'] = True

facilcost = pd.read_csv('Resources CSV - Sheet4.csv')

resources.iloc[1,0] = 'Food/Water'
resources.set_index('Resource')

hours = 0


In [0]:
def assignment(df, numberOfNormal, numberOfPilot, location, name):
  group1 = []
  count = 0
  if numberOfPilot != 0:
    count = 0
    while count < 300 and len(group1) < numberOfPilot:
        if df.loc[count, 'Working?'] == False:
          group1.append(((df.iloc[count][0]),(df.iloc[count][1])))
          df.loc[count, 'Working?'] = True
          df.loc[count, 'Where?'] = location
          count+=1
        else:
          count+=1
  if numberOfNormal != 0:
    count = 25
    while count < 300 and len(group1) < numberOfNormal + numberOfPilot:
        if df.loc[count, 'Working?'] == False:
          group1.append(((df.iloc[count][0]),(df.iloc[count][1])))
          df.loc[count, 'Working?'] = True
          df.loc[count, 'Where?'] = location
          count+=1
        else:
          count+=1
    groups[name] = group1
    return group1


In [0]:
def unassignment(df, group, location):
  group1 = groups[group]
  count = 0
  while count < len(group1):
    val = group1[count]
    val1 = val[1]
    df.loc[(df['Last Name'] == val1),'Working?'] = False 
    df.loc[(df['Last Name'] == val1),'Where?'] = location
    count+=1
  del groups[group]
  return groups
  

In [0]:
def deploy(group, item, number: int, food, med, fuel, what, location):
  if group == 'vehicles':
    vehicles.loc[(vehicles['Resource'] == item),'Number Deployed']+= number
    vehicles.loc[(vehicles['Resource'] == item),'Number Available']-= number
    if what != 'none':
      resources.loc[(resources['Resource'] == 'Food/Water'),'Available']-= food
      resources.loc[(resources['Resource'] == 'Food/Water'),'Spent']+= food
      resources.loc[(resources['Resource'] == 'Medicine units'),'Available']-= med
      resources.loc[(resources['Resource'] == 'Medicine units'),'Spent']+= med
      resources.loc[(resources['Resource'] == 'Fuel'),'Available']-= fuel
      resources.loc[(resources['Resource'] == 'Fuel'),'Spent']+= fuel
      facilities.loc[(facilities['Name/Facility'] == location),'Food/Water']+= food
      facilities.loc[(facilities['Name/Facility'] == location),'Fuel']+= fuel
      facilities.loc[(facilities['Name/Facility'] == location),'Medicine']+= med
      facilities.loc[(facilities['Name/Facility'] == location),'Running'] = True
  else:
    assets.loc[(assets['Asset'] == item),'Number Deployed']+= number
    assets.loc[(assets['Asset'] == item),'Number Available']-= number

In [0]:
def undeploy(group, item, number: int):
  if group == 'vehicles':
    vehicles.loc[(vehicles['Resource'] == item),'Number Deployed']-= number
    vehicles.loc[(vehicles['Resource'] == item),'Number Available']+= number
  else:
    assets.loc[(assets['Asset'] == item),'Number Deployed']-= number
    assets.loc[(assets['Asset'] == item),'Number Available']+= number

In [0]:
def turn():
  #hours += 6
  resources.loc[0,'Available'] -= 9500
  resources.loc[0,'Spent'] += 9500
  for item in range(0,2426):
    if facilities.loc[item,'Running'] == True:
        if facilities.loc[item,'Food/Water'] != 0:
            #print(facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'FW/Day'].sum())
            facilities.loc[item,'Food/Water'] -= facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'FW/Day'].sum()
        if facilities.loc[item,'Fuel'] != 0:
            facilities.loc[item,'Fuel'] -= facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'Fuel/Day'].sum()
        if facilities.loc[item,'Medicine'] != 0:
            facilities.loc[item,'Medicine'] -= facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'Med/Day'].sum()
        if facilities.loc[item,'Food/Water'] < facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'FW/Day'].sum():
            facilities.loc[item,'Running'] = False
        if facilities.loc[item,'Fuel'] < facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'Fuel/Day'].sum():
            facilities.loc[item,'Running'] = False
        if facilities.loc[item,'Medicine'] < facilcost.loc[(facilcost['Type'] == facilities.loc[item,'Type']),'Med/Day'].sum():
            facilities.loc[item,'Running'] = False
    
  for item in range(0,8):
      resources.loc[0,'Available'] -= vehicles.loc[item,'Money Cost Per Turn']*vehicles.loc[item,'Number Deployed']
      resources.loc[0,'Spent'] += vehicles.loc[item,'Money Cost Per Turn']*vehicles.loc[item,'Number Deployed']
  for item in range(0,3):
      resources.loc[3,'Available'] -= assets.loc[item,'Fuel Cost/Turn']*assets.loc[item,'Number Deployed']
      resources.loc[3,'Spent'] += assets.loc[item,'Fuel Cost/Turn']*assets.loc[item,'Number Deployed']
  need = []
  for i in range(0,300):
    if df.loc[i, 'Working?'] == True:
      if df.loc[i, 'Hours Worked'] < 12:
        df.loc[i, 'Hours Worked'] = df.loc[i, 'Hours Worked'] + 6
      if df.loc[i, 'Energy'] > 0:
        df.loc[i, 'Energy'] = df.loc[i, 'Energy'] - 6
      for group in groups:
        #print(groups[group][-1][0])
        if ((df.loc[i,'First Name'] == groups[group][-1][0]) and (df.loc[i, 'Energy'] == 0)):
          need.append(group)
          #unassignment(df,group,input('Where do you want group {} to rest?'.format(groups)))
        else:
          continue
  
    elif df.loc[i, 'Working?'] == False and df.loc[i, 'Energy'] < 12 :
      df.loc[i, 'Energy'] = df.loc[i, 'Energy'] + 6
      df.loc[i, 'Hours Worked'] = df.loc[i, 'Hours Worked'] - 6
  for group in need:
    unassignment(df,group,input('Where do you want group {} to rest?'.format(groups)))


In [0]:
def main():
    ask = input()
    if ask == 'turn':
        turn()
        print('6 Hours added.')
        return main()
    elif ask == 'assign':
        numbern = int(input('How many normal workers do you need?'))
        numberp = int(input('How many pilots do you need?'))
        where = input('Where are they going?')
        name = input('What is the group name?')
        assignment(df, numbern, numberp, where, name)
        print('Group assigned. 12 hours of working available.')
        return main()
    elif ask == 'unassign':
        who = input('Which group?{}'.format(groups))
        where = input('Where should they rest?')
        unassign(df,who,where)
        print('Group unassigned.')
        return main()
    elif ask == 'available':
        print(df.loc[df['Working?'] == False])
        print('Amount not working')
        print((df.loc[df['Working?'] == False]).shape[0])
        return main()
    elif ask == 'unavailable':
        print(df.loc[df['Working?'] == True])
        print('Amount working')
        print((df.loc[df['Working?'] == True]).shape[0])
        return main()
    elif ask == 'df':
        print(df)
        return main() 
    elif ask == 'groups':
        print(groups)
        return main()
    elif ask == 'end':
        print('Thank you!')
    elif ask == 'help':
        print(' \n prompts \n -------\n turn \n assign \n unassign \n available \n df \n groups \n end \n deploy \n undeploy \n vehicles \n resources \n assets \n spend \n facilities')
        return main()
    elif ask == 'deploy':
        what = input('vehicles or assets?')
        if what == 'vehicles':
            print(vehicles.loc[0:9])
        if what == 'assets':
            print(assets.loc[0:3])
        item  = input('What vehicle or asset do you need?()')
        number = int(input('How many?'))
        food = int(input('how much food/water?'))
        med = int(input('How much medecine?'))
        fuel = int(input('How much fuel?'))
        what1 = input('What type of facility is this going to?(Fire Station, Airfield, EMS, Hospital, Local Eocs, Powerplant, Shelter, State Eocs)')
        location = input('Where is the name of this place?')
        deploy(what,item,number,food,med,fuel,what1,location)
        print('Deployed.')
        return main()
    elif ask == 'undeploy':
        what = input('vehicles or assets?')
        item  = input('What vehicle or asset are you recalling?')
        number = int(input('How many?'))
        undeploy(what,item,number)
        return main()
    elif ask == 'vehicles':
        print(vehicles)
        return main()
    elif ask == 'resources':
        print(resources)
        return main()
    elif ask == 'assets':
        print(assets)
        return main()
    elif ask == 'spend':
        which = input('What resource?')
        amt = int(input('How much?'))
        resources.loc[which] -= amt 
        print(amt, 'of {} was spent.'.format(which))
        return main()
    elif ask == 'facilities':
        which = input('Which facility? (Fire Station, Airfield, EMS, Hospital, Local Eocs, Powerplant, Shelter, State Eocs)')
        if which == 'all':
            print(facilities.loc[['Name','Food/Water','Fuel','Medicine','Running']])
        else:
            try:
                print(facilities.loc[facilities['Type'] == which])
            except:
                print('Not found.')
        return main()
         
    else:
      print('Query not found.')      
      return main() 

In [0]:
resources.set_index('Resource')
path = '/content/drive/My Drive/BWSI-Remote-Sensing/Final_exercise/game1/resourceturn1.csv'
resources.to_csv(path)

In [0]:
main()

KeyboardInterrupt: ignored